# Lab 12: Rate Limiting & API Protection**Duration:** 45 minutes  **Objective:** Implement rate limiting and API protection strategies## 🎯 Learning Objectives- Implement various rate limiting algorithms- Build API quota management- Create distributed rate limiting- Implement DDoS protection- Handle throttling and backpressure

## Part 1: Rate Limiting Algorithms (10 minutes)

In [ ]:
import redisimport timeimport jsonimport hashlibfrom datetime import datetime, timedeltafrom colorama import init, Fore, Stylefrom typing import Dict, Tuple, Optionalimport mathinit(autoreset=True)# Connect to Redisr = redis.Redis(host='localhost', port=6379, decode_responses=True)r.flushdb()print(f"{Fore.GREEN}✅ Connected to Redis and cleared database{Style.RESET_ALL}")

In [ ]:
class RateLimiter:    """Implement various rate limiting algorithms."""        def __init__(self, redis_client):        self.r = redis_client        def fixed_window(self, key: str, limit: int, window: int) -> Tuple[bool, Dict]:        # Create window key based on current time        window_key = f"rate:fixed:{key}:{int(time.time()/window)}"                # Increment counter        current = self.r.incr(window_key)                # Set expiration on first request        if current == 1:            self.r.expire(window_key, window)                # Get TTL for client        ttl = self.r.ttl(window_key)                return current <= limit, {            'limit': limit,            'remaining': max(0, limit - current),            'reset': int(time.time()) + ttl        }        def sliding_window(self, key: str, limit: int, window: int) -> Tuple[bool, Dict]:        now = time.time()        window_start = now - window        window_key = f"rate:sliding:{key}"                # Remove old entries        self.r.zremrangebyscore(window_key, 0, window_start)                # Count requests in window        current = self.r.zcard(window_key)                if current < limit:            # Add current request            self.r.zadd(window_key, {str(now): now})            self.r.expire(window_key, window)            allowed = True            remaining = limit - current - 1        else:            allowed = False            remaining = 0                return allowed, {            'limit': limit,            'remaining': remaining,            'reset': int(now + window)        }        def token_bucket(self, key: str, capacity: int, refill_rate: float) -> Tuple[bool, Dict]:        bucket_key = f"rate:bucket:{key}"        now = time.time()                # Get current bucket state        bucket_data = self.r.hgetall(bucket_key)                if bucket_data:            tokens = float(bucket_data['tokens'])            last_refill = float(bucket_data['last_refill'])        else:            tokens = capacity            last_refill = now                # Calculate tokens to add        time_passed = now - last_refill        tokens_to_add = time_passed * refill_rate        tokens = min(capacity, tokens + tokens_to_add)                if tokens >= 1:            tokens -= 1            allowed = True        else:            allowed = False                # Update bucket state        self.r.hset(bucket_key, mapping={            'tokens': tokens,            'last_refill': now        })        self.r.expire(bucket_key, 3600)                return allowed, {            'tokens_remaining': int(tokens),            'capacity': capacity,            'refill_rate': refill_rate        }# Test rate limitingprint(f"{Fore.CYAN}=== RATE LIMITING ALGORITHMS ==={Style.RESET_ALL}\n")rl = RateLimiter(r)# Test fixed windowprint("Fixed Window Rate Limiting (5 requests/10 seconds):")for i in range(7):    allowed, info = rl.fixed_window("user:123", limit=5, window=10)    status = "✅ Allowed" if allowed else "❌ Blocked"    print(f"  Request {i+1}: {status} (Remaining: {info['remaining']})")    time.sleep(0.1)# Test token bucketprint(f"\n{Fore.CYAN}Token Bucket (Capacity: 5, Refill: 1/sec):{Style.RESET_ALL}")for i in range(7):    allowed, info = rl.token_bucket("user:125", capacity=5, refill_rate=1.0)    status = "✅ Allowed" if allowed else "❌ Blocked"    print(f"  Request {i+1}: {status} (Tokens: {info['tokens_remaining']})")    if i == 2:        time.sleep(2)        print("  [Waited 2 seconds for refill]")

## Part 2: API Quota Management (15 minutes)

In [ ]:
class APIQuotaManager:    """Manage API quotas and usage tracking."""        def __init__(self, redis_client):        self.r = redis_client                # Define tier limits        self.tiers = {            'free': {                'requests_per_hour': 100,                'requests_per_day': 1000,                'requests_per_month': 10000,                'burst_limit': 10            },            'basic': {                'requests_per_hour': 1000,                'requests_per_day': 10000,                'requests_per_month': 100000,                'burst_limit': 50            },            'premium': {                'requests_per_hour': 10000,                'requests_per_day': 100000,                'requests_per_month': 1000000,                'burst_limit': 200            }        }        def check_quota(self, api_key: str, tier: str = 'free') -> Tuple[bool, Dict]:        if tier not in self.tiers:            tier = 'free'                limits = self.tiers[tier]        now = datetime.now()                # Check hourly limit        hour_key = f"quota:{api_key}:hour:{now.strftime('%Y%m%d%H')}"        hour_count = int(self.r.get(hour_key) or 0)                # Check daily limit        day_key = f"quota:{api_key}:day:{now.strftime('%Y%m%d')}"        day_count = int(self.r.get(day_key) or 0)                # Check monthly limit        month_key = f"quota:{api_key}:month:{now.strftime('%Y%m')}"        month_count = int(self.r.get(month_key) or 0)                # Check burst limit        burst_key = f"quota:{api_key}:burst"        burst_count = self.r.zcount(burst_key, time.time() - 60, time.time())                # Check all limits        if (hour_count >= limits['requests_per_hour'] or            day_count >= limits['requests_per_day'] or            month_count >= limits['requests_per_month'] or            burst_count >= limits['burst_limit']):                        return False, {                'tier': tier,                'hour_usage': hour_count,                'day_usage': day_count,                'month_usage': month_count,                'burst_usage': burst_count            }                # Increment counters        pipe = self.r.pipeline()                pipe.incr(hour_key)        pipe.expire(hour_key, 3600)                pipe.incr(day_key)        pipe.expire(day_key, 86400)                pipe.incr(month_key)        pipe.expire(month_key, 2592000)                # Add to burst tracking        pipe.zadd(burst_key, {str(time.time()): time.time()})        pipe.expire(burst_key, 60)                pipe.execute()                return True, {            'tier': tier,            'hour_remaining': limits['requests_per_hour'] - hour_count - 1,            'day_remaining': limits['requests_per_day'] - day_count - 1,            'month_remaining': limits['requests_per_month'] - month_count - 1        }# Test API quotasprint(f"{Fore.CYAN}=== API QUOTA MANAGEMENT ==={Style.RESET_ALL}\n")quota_mgr = APIQuotaManager(r)# Test different tiersapi_keys = [    ('api_free_user', 'free'),    ('api_premium_user', 'premium')]for api_key, tier in api_keys:    print(f"{api_key} ({tier} tier):")        for i in range(12):        allowed, info = quota_mgr.check_quota(api_key, tier)                if i < 5 or not allowed:            status = "✅" if allowed else "❌"            if allowed:                print(f"  Request {i+1}: {status} (Hour remaining: {info.get('hour_remaining', 0)})")            else:                print(f"  Request {i+1}: {status} (Limit exceeded)")                break

## Part 3: Distributed Rate Limiting (10 minutes)

In [ ]:
class DistributedRateLimiter:    """Distributed rate limiting across multiple servers."""        def __init__(self, redis_client, node_id: str):        self.r = redis_client        self.node_id = node_id        def acquire_global_quota(self, resource: str, limit: int,                            window: int) -> Tuple[bool, Dict]:        # Use Lua script for atomic operation        lua_script = """        local key = KEYS[1]        local limit = tonumber(ARGV[1])        local window = tonumber(ARGV[2])        local now = tonumber(ARGV[3])                -- Remove old entries        redis.call('ZREMRANGEBYSCORE', key, 0, now - window)                -- Count current requests        local current = redis.call('ZCARD', key)                if current < limit then            -- Add request            redis.call('ZADD', key, now, now)            redis.call('EXPIRE', key, window)            return {1, limit - current - 1}        else            return {0, 0}        end        """                key = f"global:rate:{resource}"        now = time.time()                result = self.r.eval(lua_script, 1, key, limit, window, now)                allowed = bool(result[0])        remaining = result[1]                # Track node statistics        if allowed:            self.r.hincrby(f"node:stats:{self.node_id}", "requests", 1)        else:            self.r.hincrby(f"node:stats:{self.node_id}", "rejected", 1)                return allowed, {            'allowed': allowed,            'remaining': remaining,            'node': self.node_id        }# Test distributed rate limitingprint(f"{Fore.CYAN}=== DISTRIBUTED RATE LIMITING ==={Style.RESET_ALL}\n")# Simulate multiple nodesnodes = [    DistributedRateLimiter(r, "node-1"),    DistributedRateLimiter(r, "node-2"),    DistributedRateLimiter(r, "node-3")]print("Global Quota (10 requests/5 seconds across all nodes):")for i in range(12):    node = nodes[i % 3]    allowed, info = node.acquire_global_quota("api:endpoint", 10, 5)    status = "✅" if allowed else "❌"    print(f"  Request {i+1} from {info['node']}: {status} (Remaining: {info['remaining']})")# Show node statisticsprint(f"\n{Fore.CYAN}Node Statistics:{Style.RESET_ALL}")for i in range(3):    stats = r.hgetall(f"node:stats:node-{i+1}")    print(f"  Node-{i+1}: Requests: {stats.get('requests', 0)}, Rejected: {stats.get('rejected', 0)}")

## Part 4: DDoS Protection (10 minutes)

In [ ]:
class DDoSProtection:    """Implement DDoS protection mechanisms."""        def __init__(self, redis_client):        self.r = redis_client                # Protection thresholds        self.thresholds = {            'requests_per_second': 100,            'unique_ips_per_minute': 1000,            'requests_per_ip': 50        }        def check_ip_reputation(self, ip: str) -> Tuple[bool, str]:        # Check permanent blacklist        if self.r.sismember("blacklist:permanent", ip):            return False, "Permanently blacklisted"                # Check temporary blacklist        if self.r.exists(f"blacklist:temp:{ip}"):            return False, "Temporarily blacklisted"                # Check reputation score        reputation = self.r.zscore("ip:reputation", ip)        if reputation and reputation < -100:            return False, f"Low reputation score: {reputation}"                return True, "OK"        def detect_attack_pattern(self, ip: str, endpoint: str) -> bool:        now = time.time()                # Check rate anomaly        key = f"rate:check:{ip}:{int(now)}"        count = self.r.incr(key)        self.r.expire(key, 1)                if count > self.thresholds['requests_per_second']:            # Update reputation            self.r.zincrby("ip:reputation", -10, ip)                        # Log suspicious activity            self.r.lpush("security:suspicious", json.dumps({                'ip': ip,                'endpoint': endpoint,                'timestamp': now,                'reason': 'rate_anomaly'            }))                        return True                return False        def apply_progressive_throttling(self, ip: str) -> int:        # Get current violation level        violation_key = f"violations:{ip}"        violations = int(self.r.get(violation_key) or 0)                # Calculate delay based on violations        if violations == 0:            delay = 0        elif violations < 5:            delay = violations * 100  # 100ms per violation        elif violations < 10:            delay = violations * 500  # 500ms per violation        else:            # Temporary blacklist            self.r.setex(f"blacklist:temp:{ip}", 3600, "throttled")            delay = -1  # Blocked                return delay# Test DDoS protectionprint(f"{Fore.CYAN}=== DDoS PROTECTION ==={Style.RESET_ALL}\n")ddos = DDoSProtection(r)test_ips = [    ('192.168.1.100', 'normal'),    ('10.0.0.1', 'attacker')]for ip, behavior in test_ips:    print(f"Testing {ip} ({behavior}):")        # Check reputation    allowed, reason = ddos.check_ip_reputation(ip)    print(f"  Reputation: {'✅' if allowed else '❌'} {reason}")        if behavior == 'attacker':        # Simulate attack        for i in range(15):            is_attack = ddos.detect_attack_pattern(ip, '/api/endpoint')            if is_attack:                print(f"  ⚠️ Attack pattern detected!")                                # Apply throttling                delay = ddos.apply_progressive_throttling(ip)                if delay == -1:                    print(f"  ❌ IP temporarily blacklisted")                else:                    print(f"  ⏱️ Throttling: {delay}ms delay")                breakprint("\n✅ Rate limiting and API protection completed")